# Tests

In [25]:
import autohf as hf
import autograd.numpy as anp
import autograd
import numpy as np
from autograd.differential_operators import make_jvp
import chemistry as chem
np.set_printoptions(linewidth=300)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In this Notebook, we implement many tests for the AutoHF differentiable Hartree-Fock solver. We will be using BeH$_2$ as our reference molecule.

In [54]:
# We define some basic information about the molecule

R = anp.array([0.0, 1.0, 0.5, 0.0, -2.4359718264299817, 0.1, 0.0, 2.4359718264299817, 0.7]) # Optimal geometry
structure = ['Be', 'H', 'H']

charges = [4, 1, 1]
num_elecs = 6

R1, R2, R3 = anp.array(R[0:3]), anp.array(R[3:6]), anp.array(R[6:9]) # Atomic coordiantes

In [55]:
# We also define the basis set

basis_set = []
A1, A2, A3 = hf.basis_set_params("sto-3g", structure) # Gets default basis set parameters

for func in A1 + A2 + A3:
    L, exp, coeff = func
    basis_set.append(hf.AtomicBasisFunction(L, C=anp.array(coeff), A=anp.array(exp)))

### First Derivatives wrt Coordinates

In [56]:
def fd(func, vec, delta=0.0001):
    return lambda R : (1 / delta) * (func(R + (delta/2) * vec) - func(R - (delta/2) * vec))

In [104]:
# Testing the overlap matrix first derivative
from pennylane import qchem
num_act_orb = 6
num_act_elec = 4

core, active = qchem.active_space(6, 7, active_electrons=num_act_elec, active_orbitals=num_act_orb) # Prepares active space
fn = lambda r : hf.electron_integrals_flat(num_elecs, charges, basis_set, occupied=core, active=active)([r[0:3], r[3:6], r[6:9]], *([[r[0:3]]] * 5), [r[3:6]], [r[6:9]])
fn_jvp = make_jvp(fn)
vec = np.array([0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [105]:
import time
start = time.time()
geometry = list(zip(structure, 1/chem.data.angs_bohr * np.array([R1, R2, R3])))
wires = list(range(12))
#nuc_energy = anp.dot(autograd.grad(hf.nuclear_energy(charges))([R1, R2, R3]), vec) + core_ad
nuc_energy = 0.0
d = fn_jvp(R)(vec)[1]
core_ad, one_electron, two_electron = d[0], d[1:37].reshape((6, 6)), d[37:].reshape((6, 6, 6, 6))

dH = hf.build_h_from_integrals(geometry, one_electron, two_electron, nuc_energy, wires, basis="sto-3g", multiplicity=1, charge=0)
print(time.time() - start)

4.305045436


In [106]:
print(dH)

  (-1.6643945646781306) [I0]
+ (-0.4049369286889109) [Z2]
+ (-0.4049369286889109) [Z3]
+ (-0.226095103626895) [Z9]
+ (-0.22609510362689494) [Z8]
+ (0.033846228093047374) [Z5]
+ (0.03384622809304738) [Z4]
+ (0.047851510850894174) [Z6]
+ (0.04785151085089419) [Z7]
+ (0.08872846238762276) [Z1]
+ (0.08872846238762279) [Z0]
+ (1.100409165225626) [Z10]
+ (1.100409165225626) [Z11]
+ (-0.1370550969099844) [Z8 Z9]
+ (-0.12935957096881848) [Z2 Z3]
+ (-0.0667590931187074) [Z2 Z9]
+ (-0.0667590931187074) [Z3 Z8]
+ (-0.0525646261560924) [Z2 Z8]
+ (-0.0525646261560924) [Z3 Z9]
+ (-0.03561710344582531) [Z4 Z9]
+ (-0.03561710344582531) [Z5 Z8]
+ (-0.0352017438451646) [Z6 Z9]
+ (-0.0352017438451646) [Z7 Z8]
+ (-0.030524605097156977) [Z4 Z8]
+ (-0.030524605097156977) [Z5 Z9]
+ (-0.030296435894929735) [Z6 Z8]
+ (-0.030296435894929735) [Z7 Z9]
+ (-0.016094675244251466) [Z2 Z4]
+ (-0.016094675244251466) [Z3 Z5]
+ (-0.015810010294842625) [Z2 Z5]
+ (-0.015810010294842625) [Z3 Z4]
+ (-0.01580507601067335) [Z2

In [109]:
start = time.time()
geometry1 = list(zip(structure, 1/chem.data.angs_bohr * np.array([R1 + np.array([0.0, 0.001, 0.0]), R2, R3])))
geometry2 = list(zip(structure, 1/chem.data.angs_bohr * np.array([R1 - np.array([0.0, 0.001, 0.0]), R2, R3])))
wires = list(range(12))
nuc_energy = 0.0

d1, d2 = fn(R + np.array([0.0, 0.001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])), fn(R - np.array([0.0, 0.001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]))
core_ad1, one_electron1, two_electron1 = d1[0], d1[1:37].reshape((6, 6)), d1[37:].reshape((6, 6, 6, 6))
core_ad2, one_electron2, two_electron2 = d2[0], d2[1:37].reshape((6, 6)), d2[37:].reshape((6, 6, 6, 6))

H_up = hf.build_h_from_integrals(geometry1, one_electron1, two_electron1, 0.0, wires, basis="sto-3g", multiplicity=1, charge=0)
H_down = hf.build_h_from_integrals(geometry2, one_electron2, two_electron2, 0.0, wires, basis="sto-3g", multiplicity=1, charge=0)
H = (1 / 0.002) * chem.accelerate_add(H_up, -1 * H_down)
print(time.time() - start)

3.641558500999963


In [110]:
print(H)

  (-1.6643947588486085) [I0]
+ (-0.40493693627452476) [Z2]
+ (-0.4049369362743027) [Z3]
+ (-0.22609506342843866) [Z9]
+ (-0.22609506342820274) [Z8]
+ (0.03384622577676244) [Z4]
+ (0.033846225776915095) [Z5]
+ (0.047851466307285984) [Z7]
+ (0.04785146630735537) [Z6]
+ (0.08872852786270813) [Z1]
+ (0.08872852786272895) [Z0]
+ (1.1004092483383188) [Z10]
+ (1.1004092483383188) [Z11]
+ (-0.13705509263178817) [Z8 Z9]
+ (-0.12935957302628864) [Z2 Z3]
+ (-0.06675908590961233) [Z3 Z8]
+ (-0.06675908590961233) [Z2 Z9]
+ (-0.05256462643490889) [Z2 Z8]
+ (-0.05256462643490889) [Z3 Z9]
+ (-0.03561710399378748) [Z5 Z8]
+ (-0.03561710399378748) [Z4 Z9]
+ (-0.035201745890819924) [Z7 Z8]
+ (-0.035201745890819924) [Z6 Z9]
+ (-0.03052460640227267) [Z4 Z8]
+ (-0.03052460640227267) [Z5 Z9]
+ (-0.0302964379199841) [Z6 Z8]
+ (-0.0302964379199841) [Z7 Z9]
+ (-0.01609467640958928) [Z3 Z5]
+ (-0.01609467640958928) [Z2 Z4]
+ (-0.015810011150585446) [Z2 Z5]
+ (-0.015810011150585446) [Z3 Z4]
+ (-0.0158050780159346

In [111]:
print(chem.accelerate_add(H, -1 * dH))

  (-1.9417047791847608e-07) [I0]
+ (-4.454360820471681e-08) [Z7]
+ (-4.4543538801899984e-08) [Z6]
+ (-7.585613859273366e-09) [Z2]
+ (-7.58539181466844e-09) [Z3]
+ (-2.316284941628055e-09) [Z4]
+ (-2.3161322790232752e-09) [Z5]
+ (4.01984563414004e-08) [Z9]
+ (4.019869220828198e-08) [Z8]
+ (6.547508536736135e-08) [Z1]
+ (6.547510615628749e-08) [Z0]
+ (8.311269272809341e-08) [Z10]
+ (8.311269272809341e-08) [Z11]
+ (-3.0935683043020745e-08) [Z2 Z11]
+ (-3.0935683043020745e-08) [Z3 Z10]
+ (-1.7798481674602407e-08) [Z2 Z10]
+ (-1.7798481674602407e-08) [Z3 Z11]
+ (-1.34209800686097e-08) [Z8 Z11]
+ (-1.34209800686097e-08) [Z9 Z10]
+ (-8.799969267170304e-09) [X1 X3]
+ (-8.799969267170304e-09) [Y1 Y3]
+ (-7.628034487047741e-09) [Z3 Z6]
+ (-7.628034487047741e-09) [Z2 Z7]
+ (-7.207647160535624e-09) [X8 X10]
+ (-7.207647160535624e-09) [Y8 Y10]
+ (-4.286611940407248e-09) [Z7 Z10]
+ (-4.286611940407248e-09) [Z6 Z11]
+ (-3.4804334483884425e-09) [Z8 Z10]
+ (-3.4804334483884425e-09) [Z9 Z11]
+ (-2.56156